_Version Log: Trying to understand the theory for blaze gratings_

In [1]:
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

source= "red"
grating= "vis"
run= "1"
file_type= ".csv"
distance_to_screen = 270 #mm
grove_density = 1200 #mm^-1
blaze_angle = 36.9 #degrees
central_wavelength = 635e-9 #m 405e-9 520e-9 635e-9


In [2]:
def radians(degrees):
    return degrees*np.pi/180

def degrees(radians):
    return radians*180/np.pi

def find_files(source, grating, number, file_type):
    file_names = os.listdir("data\\")
    for file_name in file_names:
        check = re.findall('^'+source+'_'+grating+number+file_type , file_name)
        if len(check) != 0:
            new_file_name = file_name
    return new_file_name

def import_data(source, grating, number, file_type):
    file_name = find_files(source= source, grating= grating, number= number, file_type= file_type)
    if file_type == ".csv":
        data = pd.read_csv("data\\"+file_name, delimiter=",", names= ["angle", "1st order", "0th order", "-1st order"], dtype= np.float32)
    return data

In [3]:
df = import_data(source= source, grating= grating, number= run, file_type= file_type)

df["incident angle"] = df["angle"] -df["angle"][0] +45 #The first reading was done at 45 degrees
df.loc[:,"1st order":"-1st order"] = df["0th order"][0] -df.loc[:,"1st order":"-1st order"] #get the relitive positions (in mm)
df.loc[:,"1st order":"-1st order"] = degrees(np.arctan(df.loc[:,"1st order":"-1st order"] / distance_to_screen)) #the angle between the beam and the screen normal
df.loc[:,"1st order":"-1st order"] = df.loc[:,"1st order":"-1st order"].add(df["incident angle"], axis= 0) -90 #diffraction angle. By convention, the 0th order is always negative.
df.sort_values("incident angle", ignore_index= True, inplace= True)

df

,angle,1st order,0th order,-1st order,incident angle
0,86.0,NaN,NaN,-44.546082,-5.0
1,96.0,NaN,NaN,-61.392891,5.0
2,106.0,NaN,-16.709808,NaN,15.0
3,116.0,NaN,-26.340191,NaN,25.0
4,126.0,NaN,-35.994019,NaN,35.0
5,136.0,2.436646,-45.000000,NaN,45.0
6,146.0,-3.880463,-56.250507,NaN,55.0
7,156.0,-7.332359,NaN,NaN,65.0
8,166.0,-8.869514,NaN,NaN,75.0


In [4]:
def diffaction(alpha, beta, n, g): #incident angle, diffacted angle, order, grove density
    alpha = radians(np.array(alpha)) #convert to a np.array in radians
    beta = radians(np.array(beta))
    n = np.array(n)
    g *= 1e3 #convert from mm^-1 to m^-1
    sine_sum = np.sin(alpha) + np.sin(beta).T #sine_sum doesn't hold much physical significance but it is a good intermediary.
    wavelength = sine_sum.T /n /g
    return wavelength

calculated_wavelengths = diffaction(df["incident angle"], df.loc[:,"1st order":"-1st order"], [1,0,-1], grove_density)
calculated_wavelength = np.nanmean(calculated_wavelengths[np.isfinite(calculated_wavelengths)]) #find the average of the finite and valid numbers

calculated_wavelengths


C:\Users\syf83691\AppData\Local\Temp\ipykernel_8840\3929967760.py:7: RuntimeWarning: divide by zero encountered in divide
  wavelength = sine_sum.T /n /g
C:\Users\syf83691\AppData\Local\Temp\ipykernel_8840\3929967760.py:7: RuntimeWarning: invalid value encountered in divide
  wavelength = sine_sum.T /n /g


array([[           nan,            nan, 6.57198727e-07],
       [           nan,            nan, 6.58973207e-07],
       [           nan,           -inf,            nan],
       [           nan,           -inf,            nan],
       [           nan,           -inf,            nan],
       [6.24684542e-07,            nan,            nan],
       [6.26230836e-07,           -inf,            nan],
       [6.48902456e-07,            nan,            nan],
       [6.76450978e-07,            nan,            nan]])

In [5]:
percent_diff = 100 * np.abs((calculated_wavelength -central_wavelength)/central_wavelength)

print("The wavelength was calculated to be {0:.2e}m insted of {1:.2e}m which is an error of {2:.1f}%.".format(calculated_wavelength, central_wavelength, percent_diff))

The wavelength was calculated to be 6.49e-07m insted of 6.35e-07m which is an error of 2.2%.
